# Séparation et évaluation pour la planification de visites aux urgences

<div style="text-align: right;">
Mathias LOMMEL
    
<div style="text-align: right;">
Lucas OFFROY

<div style="text-align: right;">
4MA


# I/ INTRODUCTION

Dans le cadre de ce projet, comme annoncé ci-dessus, nous allons considérer le problème de planification de visites aux urgences. Nous pourrons ainsi observer 3 majeures parties :
- L'implémentation de 4 algorithmes permettant la résolution de ce problème
- L'analyse et la comparaison des performances de ces algorithmes
- L'analyse d'exemples particuliers

Ce projet nous permettra ainsi de mettre en application les notions théoriques étudiées en cours, mais aussi de prendre du recul sur nos implémentations, et se questionner sur l'aspect éthique des résultats.

In [1]:
import Pkg;
Pkg.add("GLPK")
Pkg.add("Cbc")
Pkg.add("HiGHS")
Pkg.add("JuMP")
Pkg.add("Plots")
using GLPK
using Cbc
using HiGHS
using JuMP
using Plots

    Updating registry at `C:\Users\lucas\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\lucas\.julia\environments\v1.9\Manifest.toml`


# II / Implémentation des algorithmes 

## 1 - Algorithmes de Branch and Bound

Dans cette première partie, nous allons nous pencher sur les deux premiers algorithmes de Branch and Bound, vus en TD. A cet effet, nous allons commencer par définir quelques fonctions qui nous seront utiles pour ces deux algorithmes : *estimate_sup_init* et *compute_penalties*.

In [2]:
function estimate_sup_init(P,D,H)
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
    Outputs : 
        z_sup : borne supérieure du problème
        order : ordre de passage associé à cette borne (vecteur nx1)
    """
    
    # Stratégie choisie : prendre les ratios H/P croissants
    
    # Tri des temps en fonction du critère des ratios H/P croissants
    ratios=H./P
    order=sortperm(ratios)
    
    # On décale les valeurs aux temps de commencement (ajout de 0 au debut et on retire la dernière valeur)
    temps_début_traitement_apres_patient=[0;D[order]]  
    pop!(temps_début_traitement_apres_patient)
    
    # On fait la somme cumulée pour avoir les temps de début de passage
    temps_début_traitement=cumsum(temps_début_traitement_apres_patient)
    
    # Première borne supérieure estimée
    z_sup=compute_penalties(P,H,temps_début_traitement,order) 
    
    return [z_sup,order]
end

estimate_sup_init (generic function with 1 method)

La fonction **estimate_sup_init** estime la première borne primale du problème selon le critère des ratios (Temps maximal avant pénalité)/(Pénalités) croissants. 

Ce raisonnement fait donc passer en priorité les patients qui ont une forte pénalité et un faible temps d'attente avant d'être pénalisé. Ces patients semblent en effet être à l'origine d'une forte pénalité si ils ne sont pas traités dans les premiers, ce qui justifie ce choix.

Cette première estimation est très importante notamment dans la version 2 de l'algorithme, car elle va permettre d'économiser un temps précieux de calcul. En effet, la connaissance d'une bonne borne primale permet de ne pas descendre jusqu'au bout de l'arbre, grâce à la stratégie d'élaguage.

### 1.1 : Première version

#### $\to$ Fonction *compute_penalties*

Cette fonction permet de calculer les pénalités en prenant en paramètre les vecteurs de pénalités *P*, de délais *H*, de durées de début de traitement *t*, et d'ordre provisoire *order*.

Cette fonction doit permettre de calculer les pénalités même si nous ne sommes pas dans la profondeur $n$ de l'arbre, c'est-à-dire que nous n'avons pas encore nécessairement considéré la totalité des patients dans notre vecteur *order*. Pour la suite, on normalisera le fait de mettre 0 dans les cases du tableau indicés par des nombres supérieurs au noeud en cours de traitement.

Par exemple, dans un cas de 4 patients, si on regarde le noeud de profondeur 2 où le patient 3 est traité en premier et le patient 4 est traité en 2ème, on aura $order:=[3~ 4~ 0~ 0]$. Notre fonction ne prendra donc pas en compte les zéros finaux pour calculer la pénalité (z_tilde) à ce noeud (avec la commande *order = trunc.(Int,order[order.!=0])*).

In [3]:
function compute_penalties(P,H,t,order) 
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
        t : vecteur (nx1) des durées de traitement
        order : vecteur (nx1) donnant l'ordre de passage des patients
    Output : 
        sum(res) : pénalité de l'ordre donné en paramètre
    """
    # On trie les zeros finaux si profondeur != n
    order = trunc.(Int,order[order.!=0]) 
    
    # Vecteur t associé 
    t=trunc.(Int,t[1:length(order)])  
    # Vecteur des pénalités associé
    P1=P[order]
    # Vecteur des délais associé
    H1=H[order] 
    
    # Calcul du vecteur des pénalités
    res=P1.*(t-H1) 
    # On retire les valeurs négatives
    res[res.<0].=0 
    
    # On retourne la somme de toutes les pénalités
    return sum(res) 
end

compute_penalties (generic function with 1 method)

Le vecteur *t* sera composé des durées de début de traitement, soit des durées cumulées des délais des patients traités avant. Dans l'exemple précédent, en appelant $D_3$ et $D_4$ les temps de traitement des patients 3 et 4 respectivement, *t*:=[$0$, $D_3$, $D_3+D_4$, $D_3+D_4$]. Ici, les informations intéressantes pour calculer les pénalités avec l'ordre donné sont les 2 premières cellules. Pour calculer les pénalités vectoriellement et optimiser le temps de calcul, on ne considèrera seulement ces premières cellules (avec la commande *t=trunc.(Int,t[1:length(order)]*)).
    
La taille du vecteur *order* (resp *t*) sera donc fixe de dimension $n$. Ce choix de non-recours à l'allocation dynamique est justifié par le fait que les dimensions de ce vecteur resteront petites. En effet, le nombre de patients considérés ne nécessite pas recours à l'allocation dynamique, (et serait peut être même une perte de temps au vu du nombre d'opérations nécessaires sur le vecteur *order* (resp *t*)).
    
Ensuite, on filtre P et H avec *order* pour récupérer vectoriellement les pénalités et délais dans l'ordre, et calculons vectoriellement la formule de pénalités donnée dans l'énoncé : $P.(t-H)$. 
Enfin, il ne faut pas oublier de considérer seulement les valeurs positives avec la commande *res[res.<0].=0*. 

Nous avons donc, à cette étape, le vecteur de pénalités associé à chaque patient passé dans l'ordre donné. Nous retournons donc la somme de ces pénalités pour avoir la pénalité totale calculée au noeud actuellement considéré (z_tilde).

#### $\to$ Algorithme *Branch_Bound_v1*

Nous avons construit les deux fonctions **compute_penalties** et **estimate_sup_init**, nous pouvons donc désormais construire le premier algorithme de Branch and Bound, basé sur le premier algorithme de B&B vu en TD.

In [4]:
function Branch_Bound_v1(P,D,H)
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
    Outputs : 
        z_barre : solution optimale du problème (pénalité minimale)
        ordre : ordre de passage associé (vecteur nx1)
    """

# Estimation initiale d'une borne primale, qui est renvoyée si égale à 0 car on serait dans le cas d'une optimalité
    n = length(D)
    res = estimate_sup_init(P,D,H)
    z_barre = res[1]
    ordre = res[2]
    if z_barre == 0
        return estimate_sup_init(P,D,H)
    end

# Initialisation des objet nécessaires
    penalties=0                            # Pas de pénalités au début
    temps_traitement=zeros(n)              # Pas de patients traités au début
    ordre_actuel=zeros(n)
    profondeur_actuelle=1                  # On se place à la profondeur 1 au début
    
# Boucle pour avancer dans l'arbre
    while true

        # On fait passer le patient suivant à la profondeur courante
        ordre_actuel[profondeur_actuelle]=ordre_actuel[profondeur_actuelle]+1

# Vérifications de placement dans l'arbre et de bon fonctionnement :
        
        # Si un patient est traité 2 fois (présent 2 fois dans le vecteur ordre_actuel)
        if sum(ordre_actuel.==ordre_actuel[profondeur_actuelle])>1 
            # On continue la boucle pour passer au patient suivant dans la profondeur actuelle 
            # (--> on élague)
            continue 
        end
        

        # Si tous les patients ont été traités dans la profondeur actuelle 
        ## (tous les cas de figures ont été traités à ce niveau)
        
        # ordre_actuel[profondeur_actuelle]>n signifie qu'on est passé au patient n+1 dans la profondeur actuelle  
        ## --> on sort du cadre de l'arbre
        if ordre_actuel[profondeur_actuelle]>n 
            
            if profondeur_actuelle==1
                # Si en plus, on est revenu à la profondeur de 1, on a fini de boucler sur tout notre arbre 
                ## --> on renvoie notre solution optimale
                return [z_barre,ordre]
                
            # Si nous ne sommes pas encore revenus à la profondeur 1
            else
                # On élague en remontant d'une profondeur (car toutes les branches à ce niveau ont été parcourues)
                ## On remet le compteur des patients à 0 pour la profondeur actuelle
                ordre_actuel[profondeur_actuelle]=0 
                temps_traitement[profondeur_actuelle]=0
                # On remet le calcul des pénalités à 0
                penalties=0
                # On remonte d'une profondeur
                profondeur_actuelle=profondeur_actuelle-1
                continue
            end
        end


# Vérification de l'optimalité

        # GESTION DES TEMPS DE TRAITEMENT
        
        # Si on est en profondeur 1
        if profondeur_actuelle == 1 
            # Le premier patient est traité sans délai
            temps_traitement[profondeur_actuelle]=0
        
        # Si on est plus profond dans l'arbre
        else
            # Le patient n sera traité une durée d après le patient n-1, avec d le temps de traitement du patient n-1
            ## D[trunc(Int,ordre_actuel[profondeur_actuelle-1])] donne le temps de traitement du patient précédent
            temps_traitement[profondeur_actuelle]=D[trunc(Int,ordre_actuel[profondeur_actuelle-1])]
        end
        
        # ESTIMATION DE LA BORNE DUALE
        
        # On utilise la somme cumulée des temps pour avoir le temps de passage globaux 
        ## --> conversion des temps relatifs en temps globaux
        penalties=compute_penalties(P,H,cumsum(temps_traitement),ordre_actuel) 
        
        # Si la borne duale est moins bonne que notre solution actuelle
        if penalties >= z_barre 
            # On continue la boucle pour passer au patient suivant dans la profondeur actuelle 
            ## --> on élague
            continue

        # Si au contraire, la borne duale est meilleure 
        else 

            # Si nous ne sommes pas encore arrivé au bout (tous les patients pas encore traités)
            if profondeur_actuelle != n
                # On continue sur cette branche de l'arbre (ajout d'une unité de profondeur) 
                profondeur_actuelle=profondeur_actuelle+1
                continue

            # Si nous sommes arrivés au bout de l'arbre
            else 
                # Nous avons trouvé une meilleure solution que la précédente !
                # On remplace donc l'ancienne solution par cette meilleure solution
                z_barre=penalties
                ordre=copy(ordre_actuel)
                
                # Ici, continue va nous mener à un autre tour de boucle. 
                # Forcément, vu que tous les patients ont été traités, passer au patient suivant dans la profondeur n
                # va mener à une duplication de patients dans le vecteur. Ceci va encore mener à d'autres tours 
                # de boucle jusqu'à passer au patient n+1 dans la profondeur actuelle 
                ## --> on sort du cadre de l'arbre
                
                # Ensuite, il remontera donc d'une profondeur et continuera sa course le long d'une autre branche de 
                # profondeur n-1, etc...
                
                continue
            end
        end
    end
end

Branch_Bound_v1 (generic function with 1 method)

La fonction *Branch_Bound_v1* met en oeuvre l'algorithme de branchement et séparation de la première manière étudiée en cours. L'arbre est parcouru en profondeur. 

On stocke la profondeur courante dans une variable nommée *profondeur_actuelle* et l'ordre des patients dans un vecteur nommé *ordre_actuel*. Si *profondeur_actuelle < n*, ce vecteur sera complété de 0, pour signifier que des patients dans les profondeurs *profondeur_actuelle+1, ..., n-1, n* ne sont pas encore considérés. Ainsi, au début du programme, le vecteur *ordre_actuel* sera uniquement constitué de 0. 

A chaque tour de boucle, on avance d'une unité de profondeur dans l'arbre, en considérant, à la profondeur *profondeur_actuelle +1*, le patient à l'indice suivant (+1) de celui déjà considéré à la *profondeur_actuelle*.
Si ce patient figure déjà dans la liste, on n'avance pas dans l'arbre (on n'incrémente pas la profondeur). 

Par exemple, au début du programme, on aura (pour n=4 patients) :
- 1 - ordre_actuel = [0, 0, 0, 0], profondeur_actuelle=0
- 2 - ordre_actuel = [1, 0, 0, 0], profondeur_actuelle=1
- 3 - ordre_actuel = [1, 1, 0, 0], profondeur_actuelle=2  $\to$ Doublons donc on n'avance pas de profondeur
- 4 - ordre_actuel = [1, 2, 0, 0], profondeur_actuelle=2
...etc

D'autres objets tels que : *temps_traitement* permettent de stocker les informations des temps de traitement des patients rangés dans *ordre_actuel*, dans le même ordre et décalés de 1 vers la droite (première valeur à 0 et dernière valeur à D[ordre_actuel[length(ordre_actuel)-1]]).

Ainsi, dans la fonction de calcul des pénalités, nous utiliserons la somme cummulée de ces temps, afin de connaitre les temps de passages des patients. 
Ces pénalités permettront ensuite de trouver une borne duale et/ou primale ou de juger qu'il n'est pas nécessaire d'aller plus loin dans l'arbre. 


Le programme se divise donc en 2 parties : 
- La première partie s'occupe du bon fonctionnement de la boucle décrite plus haut en vérifant que :
    - Si un patient est traité 2 fois (présent 2 fois dans le vecteur ordre_actuel), on continue la boucle pour passer au patient suivant dans la profondeur actuelle 
    
    - Si tous les patients ont été traités dans la profondeur actuelle (signifie qu'on est passé au patient n+1 dans la profondeur actuelle, donc on n'est plus dans le cadre de l'arbre $\iff$ ordre_actuel[profondeur_actuelle]>n), alors :
    
        - Si en plus, on est revenu à la profondeur de 1, on a fini de boucler sur tout notre arbre et en renvoie notre solution optimale
        - Sinon, on élague en remontant d'une profondeur (car toutes les branches à ce niveau ont été parcourues)
        
- La deuxième partie qui s'occupe de vérifier les conditions d'optimalité et qui va :
    - Mettre à jour les temps de traitement en concordance avec *ordre_actuel*
    - Calculer les pénalités
    - Décider si :
        - On élague (cas où $\widetilde{z} > \overline{z}$)
        - On continue d'explorer l'arbre en profondeur : cas où $\widetilde{z} < \overline{z}$ mais $\widetilde{x}$ pas réalisable (tous les patients n'ont pas été traités).
        - On met à jour notre solution optimale précédente : cas où $\widetilde{z} < \overline{z}$ et $\widetilde{x}$ réalisable (tous les patients ont été traités).
        
        
Le programme se termine donc en descente d'arbre comme suit (exemple avec n=4) :
- 1 - ordre_actuel = [4, 3, 2, 1], profondeur_actuelle=4   $\to$ Calcul des pénalités 
- 2 - ordre_actuel = [4, 3, 2, 2], profondeur_actuelle=4   $\to$ Doublons donc on continue
- 3 - ordre_actuel = [4, 3, 2, 3], profondeur_actuelle=4   $\to$ Doublons donc on continue
- 4 - ordre_actuel = [4, 3, 2, 4], profondeur_actuelle=4   $\to$ Doublons donc on continue
- 5 - ordre_actuel = [4, 3, 2, 5], profondeur_actuelle=4   $\to$ Sortie du cadre de l'arbre donc on remonte d'une profondeur


- 6 - ordre_actuel = [4, 3, 3, 0], profondeur_actuelle=3   $\to$ Doublons donc on continue
- 7 - ordre_actuel = [4, 3, 4, 0], profondeur_actuelle=3   $\to$ Doublons donc on continue
- 8 - ordre_actuel = [4, 3, 5, 0], profondeur_actuelle=3   $\to$ Sortie du cadre de l'arbre donc on remonte d'une profondeur


- 9 - ordre_actuel = [4, 4, 0, 0], profondeur_actuelle=2   $\to$ Doublons donc on continue
- 10 - ordre_actuel = [4, 5, 0, 0], profondeur_actuelle=2   $\to$ Sortie du cadre de l'arbre donc on remonte d'une profondeur


- 11 - ordre_actuel = [5, 0, 0, 0], profondeur_actuelle=1   $\to$ Sortie du cadre de l'arbre donc on remonte d'une profondeur


--- Fin du programme car profondeur revenue à 0 ---

### 1.2 : Seconde version

#### $\to$ Fonction *compute_penalties_v2*

Nous considérons cette fois un autre algorithme de B&B, basé sur le second raisonnement élaboré en TD. Nous devons donc repenser notre fonction *compute_penalties* : nous créons une nouvelle fonction, nommée *compute_penalties_v2*.

In [5]:
function compute_penalties_v2(P,H,D,t,order) 
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        t : vecteur (nx1) des durées de traitement
        order : vecteur (nx1) donnant l'ordre de passage des patients (inversé)
    Output : 
        sum(res) : pénalité de l'ordre donné en paramètre
    """
    # Estimation du temps total pour faire passer la totalité des patients
    max_delay=sum(D) 
    # Tri des zeros finaux si pas en profondeur n
    order = trunc.(Int,order[order.!=0])
    
    # Vecteur t associé
    t=trunc.(Int,t[1:length(order)]) 
    ###
    t= (-1).*t.+max_delay
    ###
    # Vecteur des pénalités associé
    P=P[order]
    # Vecteur des délais associés
    H=H[order]

    # Calcul du vecteur des pénalités
    res=P.*(t-H)
    # On enelève les valeurs négatives
    res[res.<0].=0

    # On retourne la somme de toutes les pénalités
    return sum(res)
end

compute_penalties_v2 (generic function with 1 method)

Cette fonction *compute_penalties_v2* permet de calculer les pénalités pour la deuxième manière de traiter notre problème : en considérant cette fois les patients traités dans l'ordre inverse de passage. 

Le principe est le même que pour la version 1, seulement, ici, la commande *t= (-1).t.+max_delay* permet de calculer les temps de passages des patients en partant de la fin. 

ATTENTION, ici, *t* sera égal aux sommes cumulées des temps de passages des patients stockés dans *ordre_actuel*. 

De cette manière, pour le patient i, on soustrait à la durée totale (temps de passage de tous les patients), les durées de passages des patients qui passent après lui. On soustrait également la durée de passage du patient i. De cette manière, on obtient bien le temps auquel passe le patient i.

Autrement, c'est le même raisonnement que pour la version 1.
La fonction permet de calculer les pénalités en prenant en paramètre les vecteurs de pénalités *P*, de délais *H*, de durées de traitement cumulées *t*, et d'ordre provisoire *order*.

#### $\to$ Algorithme *Branch_Bound_v2*

Avec cette nouvelle méthode, nous allons pouvoir écrire le second algorithme de B&B. Celui-ci fonctionnera d'une manière similaire : la seule chose qui changera sera l'ordre donné, qui sera écrit dans le sens inverse de l'ordre de passage des patients; et l'évaluation des pénalités.

In [6]:
function Branch_Bound_v2(P,D,H)
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
    Outputs : 
        z_barre : solution optimale du problème (pénalité minimale)
        reverse(ordre) : ordre de passage associé (vecteur nx1)
    """

# Estimation initiale d'une borne primale, qui est renvoyée si égale à 0 car on serait dans le cas d'une optimalité
    n=length(D)
    res=estimate_sup_init(P,D,H)
    z_barre=res[1]
    ordre=reverse(res[2]) # on reverse car on travaille à l'envers
    if z_barre == 0
        return estimate_sup_init(P,D,H)
    end
      
# Initialisation des objet nécéssaires
    penalties=0                           # Pas de pénalités au debut
    temps_traitement=zeros(n)             # Pas de patients traités au début
    ordre_actuel=zeros(n)
    profondeur_actuelle=1                 # On se place en profondeur de 1 au début

# Boucle pour avancer dans l'arbre
    while true
        
    # On fait passer le patient suivant à la profondeur courante
        ordre_actuel[profondeur_actuelle]=ordre_actuel[profondeur_actuelle]+1

# Vérifications de placement dans l'arbre et de bon fonctionnement :
                 
     # Si un patient est traité 2 fois (présent 2 fois dans le vecteur ordre_actuel)
        if sum(ordre_actuel.==ordre_actuel[profondeur_actuelle])>1 
            # On continue la boucle pour passer au patient suivant dans la profondeur actuelle 
            ## --> on élague
            continue 
        end
        
        # Si tous les patients ont été traités dans la profondeur actuelle 
        ## --> tous les cas de figures ont été traités à ce niveau
        
        # ordre_actuel[profondeur_actuelle]>n signifie qu'on est passé au patient n+1 dans la profondeur actuelle donc 
        # on n'est plus dans le cadre de l'arbre
        if ordre_actuel[profondeur_actuelle]>n 
            
            # Si en plus, on est revenu à la profondeur 1, on a fini de boucler sur tout notre arbre 
            ## --> on renvoie notre solution optimale
            if profondeur_actuelle==1
                return [z_barre,reverse(ordre)]
            
            # Si nous ne sommes pas encore revenus à la profondeur 1
            else
                # On élague en remontant d'une profondeur (car toutes les branches à ce niveau ont été parcourues)
                # On remet le compteur des patients à 0 pour la profondeur actuelle
                ordre_actuel[profondeur_actuelle]=0
                temps_traitement[profondeur_actuelle]=0
                # On remet le calcul des pénalités à 0
                penalties=0
                # On remonte d'une profondeur
                profondeur_actuelle=profondeur_actuelle-1
                continue
            end
        end

# Vérification de l'optimalité
        
        # GESTION DES TEMPS DE TRAITEMENT
        
        # Dans cette version, les temps_traitement sont égaux aux temps de passages des patients stockés dans ordre_actuel
        ## --> On donne à la fonction compute_penalties_v2 les sommes cumulées des temps_traitement

        temps_traitement[profondeur_actuelle]=D[trunc(Int,ordre_actuel[profondeur_actuelle])]
        
        # Estimation des pénalités au niveau de profondeur actuel (borne duale)
        penalties=compute_penalties_v2(P,H,D,cumsum(temps_traitement),ordre_actuel)
        

        # Si la borne duale est moins bonne que notre solution actuelle
        if penalties >= z_barre 
            # On continue le bouclage pour passer au patient suivant dans la profondeur actuelle 
            ## --> on élague
            continue
            
        # Si, au contraire, la borne duale est meilleure 
        else 
            
            # Si nous ne sommes pas encore arrivé au bout (tous les patients ne sont pas encore traités)
            if profondeur_actuelle != n
                # On continue sur cette branche de l'arbre (ajout d'une unité de profondeur) 
                profondeur_actuelle=profondeur_actuelle+1
                continue

            # Si nous sommes arrivés au bout de l'arbre
            else 
                # Nous avons trouvé une meilleure solution que la précédente !
                # On remplace donc l'ancienne solution par cette meilleure solution
                z_barre=penalties
                ordre=copy(ordre_actuel)
                
                # Ici, "continue" va nous mener à un autre tour de boucle. 
                
                # Forcément, vu que tous les patients ont été traités, passer au patient suivant dans la profondeur n
                ## va mener à une duplication de patients dans le vecteur. 
                
                # Ceci va encore mener à d'autres tours de boucle jusqu'à passer au patient n+1 dans la profondeur 
                ## actuelle (on ne sera plus dans le cadre de l'arbre).
                
                # Ensuite, il remontera donc d'une profondeur et continuera sa course le long d'une autre branche de 
                # profondeur n-1, etc...
                
            end
        end
    end
end


Branch_Bound_v2 (generic function with 1 method)

Cette deuxième version fonctionne ainsi de la même manière que la première, à la différence que les pénalités sont calculées autrement (comme expliqué plus haut).

Cela implique donc que le vecteur *temps_traitement* est traité autrement, en stockant cette fois, à l'indice i, la durée de traitement du patient placé à l'indice i du vecteur *ordre_actuel*. 

On fait également appel à la fonction *compute_penalties_v2* et non plus à la fonction *compute_penalties_v1*.

## 2 - Formulation PLNE

Dans le cadre de ce 3e algorithme, l'objectif va être d'écrire le problème considéré en PLNE. Nous pourrons ensuite utiliser une résolution de ce problème linéaire en nombres entiers, à l'aide d'un solveur tel que HiGHS.

Pour cette formulation, nous allons utiliser 3 variables différentes : 

- $x_{i,j} \in \left\{0,1 \right\} = 1 \iff \textit{ le patient } i \textit{ passe avant le patient j, } ~~\forall~ i,j \in \left\{1,2,...,n \right\}$

- $r_i \ge 0 \textit{ : correspond au retard du patient i par rapport à sa date prévue au pire}$

- $t_i \ge 0 \textit{ : correspond à l'instant de passage du patient i}$


Les contraintes considérées pour ces variables sont les suivantes :

- la variable $x_{i,j}$ traduit un ordre total, nous devons donc considérer deux contraintes :  

    Transitivité : si le patient i passe avant le patient j, et le patient j passe avant le patient k, alors le patient i passe avant le patient k
    
$$\forall~i,j,k \in \left\{1,...,n \right\}, ~~ i \neq j, i \neq k, j \neq k,~~ x_{i,k} \ge x_{i,j}+x_{j,k}-1$$ 

    Ordre total : soit le patient i passe avant le patient j, soit j passe avant i

$$\forall~i,j \in \left\{1,...,n \right\}, ~~ i \neq j,~~ x_{i,j} + x_{j,i}=1~~,~ et~~ \forall i \in \left\{1,...,n \right\},~~ x_{i,i}=0$$

- la variable $t_i$ correspond à l'instant de passage du patient i : cet instant correspond à la somme des durées de rdv de chaque patient passé avant lui
$$\forall i \in \left\{1,...,n \right\},~~ t_i = \sum_{j = 1}^n x_{j,i}.D_j$$

- la variable $r_i$ correspond au retard par rapport à l'instant de passage au pire du patient : $r_i = max(0,t_i-H_i)$. Cela se traduit avec deux contraintes (uniquement 2 car nous souhaitons ici minimiser ces retards) :
$$\forall i \in \left\{1,...,n \right\},~~ r_i \ge 0 ~~,~et~~ r_i \ge t_i-H_i$$


Bien entendu, dans ce problème, l'objectif est de minimiser la pénalité engendrée par l'ordre de passage des patients, la fonction objectif s'écrit donc : $$min \sum_{i=1}^n r_i.P_i$$


Notre problème est donc défini, et nous allons pouvoir écrire une méthode *PLNE*, permettant de construire un tel modèle.

In [7]:
function PLNE(P,D,H)
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
    Output : 
        model : modèle décrit ci-dessus
    """
    # HiGHS solver
    model = Model(HiGHS.Optimizer); 
    # On décide de ne pas afficher les outputs du modèle
    set_optimizer_attribute(model, "output_flag", false) 
    
    # Variables
    n=length(P)
    I=Array(1:n)
        
    ## r[i] = retard du patient i par rapport à sa date prévue au pire
    @variable(model, r[I]>=0);
    
    ## t[i] = instant de passage du patient i
    @variable(model, t[I]>=0);
    
    ## x[i,j] = 1 ssi le patient i passe avant le patient j
    @variable(model, x[I,I], Bin);

    # Fonction objectif : minimiser les pénalités liés aux retards
    @objective(model, Min, sum(r[i]*P[i] for i in I));
    
    # Contraintes
    
    ## Contraintes liées à l'ordre total x[i,j]
    ### Contrainte de transitivité
    @constraint(model, [i in I, j in I, k in I; i!=j && i!=k && j!=k], x[i,k]>=x[i,j]+x[j,k]-1)
    ### Ordre total
    @constraint(model, [i in I, j in I; i!=j], x[i,j]==1-x[j,i])
    
    ## Contrainte liée à l'instant de passage
    @constraint(model, [i in I], t[i]==sum(D[j]*x[j,i] for j in I))
    
    ## Contraintes liées au retard mesuré
    @constraint(model, [i in I], r[i]>=t[i]-H[i])

    return model;
end

PLNE (generic function with 1 method)

Ensuite, nous pouvons écrire une seconde fonction, qui résout le modèle, et détermine l'ordre de passage des patients.

Pour déterminer cet ordre de passage, nous allons utiliser les instants de passages de chaque individu (variable $t$), et nous allons récupérer l'ordre correspondant à l'ordonnancement croissant de ces instants.

In [8]:
function solve_PLNE(model)
    """
    Input :
        model : modèle à résoudre
    Outputs : 
        terminaton_status(model) : état de la solution 
        objective_value(model) : valeur optimale de la solution
        indices : ordre associé à la solution
    """
    # On résout le PLNE
    optimize!(model); 
    # On récupère l'ordre de passage des patients
    t=value.(model[:t])
    T=convert(Array, t)
    indices=sortperm(T)
    
    return ([termination_status(model), objective_value(model),indices]);
end

solve_PLNE (generic function with 1 method)

## 3 - Branch & Bound basé sur la relaxation linéaire

Nous allons, dans cette partie, implémenter un 3e algorithme de B&B.

Celui-ci est assez différent des deux premières implémentations de type B&B. En effet, ici, nous allons nous baser sur la relaxation linéaire de la formulation en PLNE vue précédemment.

L'idée est donc la suivante : 
- on calcule une borne supérieure $\overline{z}$ du problème, et on garde en mémoire l'ordre associé
- on construit le modèle global, sans contrainte supplémentaire : on résout alors sa relaxation linéaire pour obtenir une borne inférieure $\widetilde{z}$.

Plusieurs cas de figure peuvent être rencontrés : 
- $\widetilde{z} \ge \overline{z}$ : l'ordre trouvé est déjà optimal
- $\widetilde{z} < \overline{z}$ :
    - si la solution $\widetilde{x}$ est réalisable (dans $\mathbb{Z}$), on met à jour la borne supérieure ($\overline{z} \gets \widetilde{z}$ et on met à jour l'ordre associé, puis on élague)
    - sinon, on va descendre dans l'arbre : 
        - on détermine la composante de $\widetilde{x}_{i,j}$ la plus fractionnaire (la plus proche de 0.5)
        - on crée deux noeuds fils, avec deux contraintes supplémentaires disjointes : $x_{i,j} \ge 1$ et $x_{i,j} \le 0$
        - on se déplace vers le noeud caractérisé par $x_{i,j} \le 0$, on construit le modèle associé, avec cette nouvelle contrainte, et on met à jour $\widetilde{z}$, $\widetilde{x}$
        
Arrivé à ce noeud, on réalise un raisonnement analogue au précédent :
- $\widetilde{z} \ge \overline{z}$ : on élague (on remonte dans l'arbre)
- $\widetilde{z} < \overline{z}$ :
    - si la solution $\widetilde{x}$ est réalisable (dans $\mathbb{Z}$), on met à jour la borne supérieure ($\overline{z} \gets \widetilde{z}$ et on met à jour l'ordre associé, puis on élague : on remonte dans l'arbre)
    - sinon, on va descendre dans l'arbre : 
        - on détermine la composante de $\widetilde{x}_{i,j}$ la plus fractionnaire (la plus proche de 0.5)
        - on crée deux noeuds fils, avec une contrainte supplémentaire : $x_{i,j} \ge 1$ ou $x_{i,j} \le 0$
        - on se déplace vers le noeud caractérisé par $x_{i,j} \le 0$, on construit le modèle associé, avec cette nouvelle contrainte, et on met à jour $\widetilde{z}$, $\widetilde{x}$
        
Et ainsi de suite...


Ainsi, comme pour les deux B&B précédents, nous opterons pour un parcours en profondeur. Ce choix réside principalement dans le fait que ce type de parcours nous permet de se déplacer dans l'arbre sans devoir garder en mémoire chaque modèle construit à chaque niveau de l'arbre.

#### $\to$ Fonction *generate_model*

Nous commençons par construire une méthode permettant de construire le modèle considéré à chaque noeud. Celle-ci prend 4 paramètres : P,D,H (utiles pour construire le modèle de base), et un vecteur *contraintes*, contenant les contraintes ajoutées au modèle initial.

Ces contraintes seront codées de la manière suivante :  
- pour une contrainte $x_{i,j} \ge 1$, la contrainte vaudra [i ,j , 1]
- pour une contrainte $x_{i,j} \le 0$, la contrainte vaudra [i ,j , 0]

Dans le nouveau B&B, nous créerons un vecteur de contraintes (nommé *contraintes_actuelles*), contenant toutes les contraintes supplémentaires du modèle initial. Nous ajouterons ainsi de manière itérative chaque contrainte supplémentaire à la fin de ce vecteur.

In [9]:
function generate_model(P,D,H,contraintes)
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
        contraintes : vecteur contenant les contraintes supplémentaires
    Output : 
        model : modèle construit
    """
    # HiGHS solver
    model = Model(HiGHS.Optimizer); 
    # On décide de ne pas afficher les outputs du modèle
    set_optimizer_attribute(model, "output_flag", false) 
    
    # Variables
    n=length(P)
    I=Array(1:n)
        
    ## r[i] = retard du patient i par rapport à sa date prévue au pire
    @variable(model, r[I]>=0);
    
    ## t[i] = instant de passage du patient i
    @variable(model, t[I]>=0);
    
    ## x[i,j] = 1 ssi le patient i passe avant le patient j
    @variable(model, x[I,I], Bin);

    # Fonction objectif : minimiser les pénalités liés aux retards
    @objective(model, Min, sum(r[i]*P[i] for i in I));
    
    # Contraintes
    
    ## Contraintes liées à l'ordre total x[i,j]
    ### Contrainte de transitivité
    @constraint(model, [i in I, j in I, k in I; i!=j && i!=k && j!=k], x[i,k]>=x[i,j]+x[j,k]-1)
    ### Ordre total
    @constraint(model, [i in I, j in I; i!=j], x[i,j]==1-x[j,i])
    
    ## Contrainte liée à l'instant de passage
    @constraint(model, [i in I], t[i]==sum(D[j]*x[j,i] for j in I))
    
    ## Contraintes liées au retard mesuré
    @constraint(model, [i in I], r[i]>=t[i]-H[i])
    
    ## On ajoute les contraintes supplémentaires
    for c in contraintes
        @constraint(model, x[c[1],c[2]]==c[3])
    end

    return model;
end

generate_model (generic function with 1 method)

Nous construisons également une fonction *solve_relaxation*, qui sera utilisée à chaque noeud de l'arbre. Celle-ci résout la relaxation linéaire du modèle considéré.

In [10]:
function solve_relaxation(model)
    """
    Input :
        model : modèle dont on veut résoudre la relaxation
    Output : 
        model : relaxation du modèle (résolue)
    """
    # On résout la relaxation
    undo = relax_integrality(model);
    set_optimizer_attribute(model, "output_flag", false) # mute solvers' outputs 
    optimize!(model); 
    # On renvoie le modèle
    return model;
end

solve_relaxation (generic function with 1 method)

#### $\to$ Fonctions *getOrdre* et *getX*

A chaque noeud de notre arbre, nous devons récupérer la matrice $\widetilde{X}$, solution de la relaxation du noeud considéré. C'est pourquoi nous créons une fonction *getX*, permettant de récupérer cette variable pour un modèle donné.

Dans cette fonction, nous testons également si la matrice obtenue est dans $\mathbb{Z}$ (si $\forall~i,j \in \left\{1,...,n \right\},~~x_{i,j} \in \left\{ 0,1 \right\}$). Cette information sera contenue dans la variable *dans_Z*.

In [11]:
function getX(model)
    """
    Input :
        model : modèle relaxé résolu dont on souhaite récupérer la solution
    Outputs : 
        X : matrice X du modèle 
        dans_Z : variable booléenne à 1 si toutes les composantes de X sont binaires
    """
    # On récupère la variable X
    tab=value.(model[:x])
    X=convert(Array, tab)
    # On teste si la solution est réalisable (x_{i,j}=0 ou x_{i,j}=1 pour tout i,j)
    dans_Z=all(elements->isinteger(elements),X)
    return([X,dans_Z])   
end

getX (generic function with 1 method)

Dans le cas où la solution de la relaxation est réalisable, nous devons récupérer l'ordre associé à cette solution. Nous devons donc déterminer l'ordre de passage des patients pour un modèle donné.

Comme nous avions pu faire dans la partie précédente, nous pouvons récupérer cet ordre en récupérant les instants de passage de chaque patient.

In [12]:
function getOrdre(model)
    """
    Input :
        model : modèle relaxé dont on souhaite déterminer l'ordre
    Output : 
        indices : ordre de passage
    """
    # On récupère les instants de passage
    t=value.(model[:t])
    T=convert(Array, t)
    # On récupère l'ordre de passage
    indices=sortperm(T)
    return(indices)   
end

getOrdre (generic function with 1 method)

#### $\to$ Algorithme *Branch_Bound_v3*

In [13]:
function Branch_Bound_v3(P,D,H)
    """
    Inputs :
        P : vecteur (nx1) des pénalités associées au retard de chaque patient
        D : vecteur (nx1) des durées de rdv de chaque patient
        H : vecteur (nx1) des horaires de passage au plus tard de chaque patient
    Output : 
        z_state : état de la solution (rendu uniquement si au moins une relaxation est réalisée)
        z_barre : solution du problème
        ordre : ordre de passage de la solution
    """
    # Nombre de patients
    n=length(D)
    
    # Détermination d'une borne primale
    res=estimate_sup_init(P,D,H)
    z_barre=res[1]
    ordre=res[2]
    if z_barre==0
        # Si la borne supérieure est nulle, elle est optimale --> on renvoie la solution
        return [z_barre,ordre]
    end
    # Détermination d'une borne duale
    modeleCourant=generate_model(P,D,H,[])         # On initialise le modèle en PLNE sans contraintes supplémentaires
    solve_relaxation(modeleCourant)                # On résout la relaxation linéaire de ce PLNE
    z_tilde=objective_value(modeleCourant)         # Récupération de la borne inférieure
    z_state=termination_status(modeleCourant)      # Etat de la solution
    getteur=getX(modeleCourant)                    # Récupération de la matrice X associée
    x_tilde=getteur[1]
    dans_Z=getteur[2]                              # Indicateur donnant si X est dans Z ou non
    
    
    elague=false    # Variable binaire : à un noeud donné, =0 si on branche / =1 si on élague
    profondeur=1    # Profondeur de parcours de l'arbre
    
    # Tableau gardant en mémoire les contraintes 
    contraintes_actuelles=[]         # Contraintes du modèle courant
    
    if z_barre==z_tilde 
        # Cas où la borne supérieure est déjà optimale / borne primale nulle
        return [z_barre,ordre]
    else 
        # Dans le cas contraire, on commence le parcours de l'arbre
        while(true)
            if (z_barre<=z_tilde)
                # Cas où la relaxation donne une borne inférieure plus grande que z_barre --> On élague
                elague=true
            end 
            if (dans_Z)
                # Cas où le x_tilde a toutes ses composantes entières (solution réalisable)
                if (z_tilde < z_barre)
                    #  si la valeur optimale est meilleure : on met à jour z_barre 
                    z_barre=copy(z_tilde)
                    x_barre=copy(x_tilde)
                    ordre=getOrdre(modeleCourant)
                end
                # Dans tous les cas, on élague
                elague = true 
            end
            
            
            if (!elague)
                # Cas où x_tilde est non réalisable pour Pk
                ##  --> On recherche la composante la plus fractionnaire
                prox=abs.(x_tilde.-0.5)
                indices=argmin(prox)
                i=indices[1]
                j=indices[2]

                # On détermine la contrainte à ajouter au modèle
                ## On code la contrainte avec 2 paramètres : 
                ##         - (i,j) : indices de la composante la plus fractionnaire
                ##         - k (=0 ou =1): contrainte x_ij == k
                nouvelleContrainte=[i,j,0]
                
                # On ajoute la contrainte à la liste des contraintes traitées
                push!(contraintes_actuelles,nouvelleContrainte)

                # Modification du modèle
                modeleCourant=generate_model(P,D,H,contraintes_actuelles)   # On ajoute la contrainte au modèle
                
                # On a ici ajouté une contrainte : on descend d'un rang dans l'arbre
                profondeur=profondeur+1
            
            else
                # On élague le noeud
                elague=false     
                
                # On remonte jusqu'au premier noeud pas encore traité selon un parcours en profondeur
                noeud_trouve=false          # Devient "false" lorsque l'on se trouve à un nouveau noeud
                
                while(!noeud_trouve)
                    # Tant qu'un tel noeud n'est pas trouvé
                    
                    # On remonte d'un niveau
                    profondeur=profondeur-1
                    # On supprime la dernière contrainte ajoutée
                    contrainte_a_supprimer=contraintes_actuelles[length(contraintes_actuelles)]  
                    deleteat!(contraintes_actuelles, findall(x->x==contrainte_a_supprimer,contraintes_actuelles))
                    
                    # On détermine la contrainte complémentaire de celle supprimée
                    contrainteC=copy(contrainte_a_supprimer)
                    contrainteC[3]=abs(1-contrainteC[3])
                    
                    # Si la contrainte complémentaire n'a pas été traitée
                    if (contrainteC[3]==1)
                        # On ajoute la contrainte complémentaire aux contraintes traitées
                        push!(contraintes_actuelles,contrainteC)
                        
                        # On l'ajoute au modèle
                        modeleCourant=generate_model(P,D,H,contraintes_actuelles)
                        
                        # On est redescendu dans l'arbre : on sort de la boucle while et on incrémente la profondeur
                        noeud_trouve=true
                        profondeur=profondeur+1  
                    end
                    
                    if !noeud_trouve && profondeur==1
                        # On est remonté au probleme initial 
                        ## --> tous les cas ont été traités : on renvoie z_barre et ordre
                        return [z_state, z_barre, ordre]
                    end
    
                end
            end
            # On résout la relaxation de Pk
            solve_relaxation(modeleCourant)
            # On met à jour la solution relaxée
            z_tilde=objective_value(modeleCourant)         # Récupération de la borne inférieure
            z_state=termination_status(modeleCourant)      # Etat de la solution
            getteur=getX(modeleCourant)                    # Récupération de la matrice X associée
            x_tilde=getteur[1]
            dans_Z=getteur[2]                              # Indicateur donnant si X est dans Z ou non
        end
    end  
end

Branch_Bound_v3 (generic function with 1 method)

#### $\to$ Fonction *printBB*

Définition d'un fonction d'affichage des solutions, qui permet de se concentrer sur l'ordre de passage de certains patients (en les affichant en rouge), ou d'afficher tous les patients d'une différente couleur, dans l'ordre, afin de pouvoir comparer les résultats de différentes instances visuellement plus facilement.

In [14]:
function printBB(a, patients_remarquables=[])
    """
    Inputs : 
        a : vecteur rendu par l'algorithme qui résout le problème
        patients_remarquables : patients que l'on souhaite étudier    
    """
    
    if a[1] == OPTIMAL
        penalite = trunc(Int,a[2])
        ordre = a[3]
    else
        penalite = trunc(Int,a[1])
        ordre = a[2]
    end
        
    print("Pénalités : ")
    println(penalite)
    println("Ordre de passage :")
    
    if length(patients_remarquables)==0
        for i in 1:length(ordre) 
            printstyled(i,"-->";color = :black)
            printstyled(" Patient ",trunc(Int,ordre[i]),"\n"; color = trunc(Int,ordre[i]+1))
        end
        
        
    else 
        for i in 1:length(ordre) 
            if ordre[i] in patients_remarquables 
                col = :red
            else 
                col= :blue
            end
            printstyled(i," -->";color = :black)
            printstyled(" Patient ",trunc(Int,ordre[i]),"\n"; color = col)
        end
    end
end

# Exemple :
P=[5,4,1,2]
D=[14,15,16,13]
H=[1,2,3,4]
## Impression de l'ordre
printBB(Branch_Bound_v2(P,D,H))
print("\n")
## Impression de l'ordre en se concentrant sur le patient 4
printBB(Branch_Bound_v2(P,D,H),[4])


Pénalités : 137
Ordre de passage :
1--> Patient 1
2--> Patient 2
3--> Patient 4
4--> Patient 3

Pénalités : 137
Ordre de passage :
1 --> Patient 1
2 --> Patient 2
3 --> Patient 4
4 --> Patient 3


Cette fonction sera très utile, notamment dans la dernière partie de ce projet.